### After performing EDA, it's now time to create features using transactional data both historical and new as well as merchant information
* Let's do some preliminary analysis on the number of cards and how to merge various groups of data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import eda_fe_module as eda_fe
import pickle
from tqdm import tqdm
import time

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [24]:
train_card_ids = train_df['card_id'].nunique()
test_card_ids = test_df['card_id'].nunique()

print("Number of card_id's in train_data : ",train_df['card_id'].nunique())
print("Number of card_id's in test_data : ",test_df['card_id'].nunique())
print("Total number of card_id's : ",train_card_ids + test_card_ids)

Number of card_id's in train_data :  201917
Number of card_id's in test_data :  123623
Total number of card_id's :  325540


In [13]:
hist_trans_df = eda_fe.reduce_mem_usage(pd.read_csv('historical_transactions.csv'))
new_trans_df = eda_fe.reduce_mem_usage(pd.read_csv('new_merchant_transactions.csv'))

hist_trans_df = eda_fe.reduce_mem_usage(hist_trans_df[['card_id']])
new_trans_df = eda_fe.reduce_mem_usage(new_trans_df[['card_id']])

Mem. usage decreased to 1749.11 Mb (43.7% reduction)


In [10]:
print("Number of unique card_ids in historical_transactions : ",hist_trans_df['card_id'].nunique())
print("Number of unique card_ids in new transactions : ",new_trans_df['card_id'].nunique())

Number of unique card_ids in historical_transactions :  325540
Number of unique card_ids in new transactions :  290001


In [14]:
hist_new_card_intersect = len(set(hist_trans_df['card_id'].unique()).intersection(set(new_trans_df['card_id'].unique())))
train_hist_card_intersect = len(set(hist_trans_df['card_id'].unique()).intersection(set(train_df['card_id'].unique())))
test_hist_card_intersect = len(set(hist_trans_df['card_id'].unique()).intersection(set(test_df['card_id'].unique())))
train_test_card_intersect = len(set(train_df['card_id'].unique()).intersection(set(test_df['card_id'].unique())))

In [25]:
hist_new_card_intersect

290001

### Observation
* All card_ids under new merchant transactions are a subset of historical transactions.
* While there are some card ids that are exclusively present in historical transactions that belong to train data.
* New Merchant transactions don't contain all of the train data card_ids but contain all of the test card_ids.

In [15]:
print("Train Test card intersection : ",train_test_card_intersect)

Train Test card intersection :  0


In [17]:
print("Test Hist card intersection : ",test_hist_card_intersect)
print("Train Hist card intersection : ",train_hist_card_intersect)

Test Hist card intersection :  123623
Train Hist card intersection :  201917


In [18]:
train_new_card_intersect = len(set(new_trans_df['card_id'].unique()).intersection(set(train_df['card_id'].unique())))
test_new_card_intersect = len(set(new_trans_df['card_id'].unique()).intersection(set(test_df['card_id'].unique())))

In [19]:
print("Train New card intersection : ",train_new_card_intersect)
print("Test New card intersection : ",test_new_card_intersect)

Test New card intersection :  179986
Train New card intersection :  110015


## Approach after analysis 

### Step-1
* There is no overlap between train and test dataframe card_id's.
* We have to sample data from historical and new merchant transactions that belong only to train_df.
* Apply all of the feature transformation to train_df.
    * Feature transformation or aggregation on historical_transactions.
    * Feature transformation or aggregation on new_merchant_transactions.
    * Feature transformation on merchants data.
    * Merge train_df, hist_aggr_df, new_merch_aggr_df and merchants_aggr_df.
* Do feature selection using transformed train_df.

### Step-2
* Create a cross-validation framework for thorough training of models.
* Perform train and cross-validation split using transformed train_df.
* Keep on stacking models to create a better model in each iteration.
* Apply feature transformation on test_df,historical_transactions and new_merchant_transactions based on card_ids from test_df.
* Test your model's performance using test_df.

### Step-3
* Once confident with the model test your data against sample_submission.csv

In [21]:
new_trans_df = eda_fe.reduce_mem_usage(pd.read_csv('new_merchant_transactions.csv'))

Mem. usage decreased to 114.20 Mb (45.5% reduction)


In [22]:
new_trans_df.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_415bb3a509,107,N,1,B,307,M_ID_b0c793002c,1,-0.557617,2018-03-11 14:57:36,1.0,9,19
1,Y,C_ID_415bb3a509,140,N,1,B,307,M_ID_88920c89e8,1,-0.569336,2018-03-19 18:53:37,1.0,9,19
2,Y,C_ID_415bb3a509,330,N,1,B,507,M_ID_ad5237ef6b,2,-0.551270,2018-04-26 14:08:44,1.0,9,14
3,Y,C_ID_415bb3a509,-1,Y,1,B,661,M_ID_9e84cda3b1,1,-0.671875,2018-03-07 09:43:21,NaN,-1,8
4,Y,C_ID_ef55cf8d4b,-1,Y,1,B,166,M_ID_3c86fa3831,1,-0.659668,2018-03-22 21:07:53,NaN,-1,29


## 1. Objective  : Feature Transformation on train data
* Based on our observations in train data we are building these features.
* We are creating trans_(csv_file)_file for storing our features generated from each of the files. 

In [4]:
trans_train_df = train_df.copy()
trans_train_df['feature_2_bin'] = train_df['feature_2'].apply(lambda x : 1 if x==3 else 0)
trans_train_df['feature_1_bin'] = train_df['feature_1'].apply(lambda x : 1 if x>=3 else 0)
trans_train_df['first_active_month'] = pd.to_datetime(train_df['first_active_month'])
trans_train_df['first_active_day'] = trans_train_df['first_active_month'].dt.day
trans_train_df['first_active_wday'] = trans_train_df['first_active_month'].dt.weekday
trans_train_df['first_active_mm'] = trans_train_df['first_active_month'].dt.month
trans_train_df['first_active_year'] = trans_train_df['first_active_month'].dt.year

In [5]:
trans_train_df.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,feature_2_bin,feature_1_bin,first_active_day,first_active_wday,first_active_mm,first_active_year
0,2017-06-01,C_ID_92a2005557,5,2,1,-0.820283,0,1,1,3,6,2017
1,2017-01-01,C_ID_3d0044924f,4,1,0,0.392913,0,1,1,6,1,2017
2,2016-08-01,C_ID_d639edf6cd,2,2,0,0.688056,0,0,1,0,8,2016
3,2017-09-01,C_ID_186d6a6901,4,3,0,0.142495,1,1,1,4,9,2017
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,-0.159749,1,0,1,2,11,2017


In [6]:
trans_train_df.columns

Index(['first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3',
       'target', 'feature_2_bin', 'feature_1_bin', 'first_active_day',
       'first_active_wday', 'first_active_mm', 'first_active_year'],
      dtype='object')

In [7]:
trans_train_df.shape

(201917, 12)

## 1.1 Objective : Collect all the unique card_ids for sampling train card_ids from historical transactions and new merchant transactions

In [11]:
train_card_ids = trans_train_df['card_id'].unique()

## Save card_ids
pickle.dump(train_card_ids,open('train_card_ids.pkl','wb'))

## train_df feature set
pickle.dump(trans_train_df,open('trans_train_df.pkl','wb'))

## 2 Objective : Sample historical_transactions based on card_ids in train data  
* We are creating sampled historical transactions data based on card_ids in train data.
* Also we are adding the new purchase amount field based on our eda.

In [14]:
size_of_hist_df = hist_trans_df.shape[0]
hist_trans_df = hist_trans_df.loc[hist_trans_df['card_id'].isin(train_card_ids)]
size_of_s_hist_df = hist_trans_df.shape[0]
print("Percentage of data sampled : ",size_of_s_hist_df*100/size_of_hist_df)

Percentage of data sampled :  61.932486341454755


In [16]:
pickle.dump(hist_trans_df,open('sampled_hist_df.pkl','wb'))

In [3]:
s_hist_df = pickle.load(open('sampled_hist_df.pkl','rb'))

In [4]:
s_hist_df['new_purchase_amount'] = np.round(s_hist_df['purchase_amount'] / 0.00150265118 + 497.06,2)

## 2.1 Objective : To observe percentage of null values and fill them

In [3]:
s_hist_df.isnull().sum()*100/s_hist_df.shape[0]

authorized_flag         0.000000
card_id                 0.000000
city_id                 0.000000
category_1              0.000000
installments            0.000000
category_3              0.612939
merchant_category_id    0.000000
merchant_id             0.477132
month_lag               0.000000
purchase_amount         0.000000
purchase_date           0.000000
category_2              9.081482
state_id                0.000000
subsector_id            0.000000
dtype: float64

## Analysis
* Let's replace category_2 and category_3 by max occuring category_value.
* Let's not replace merchant_id because apart from matching with merchant transaction data we are not going use it.
* Or Let's just replace it with not_known.

In [10]:
## Replacing category_3 by max occuring category
max_cat = s_hist_df['category_3'].value_counts().index[0]
print("Most frequently occuring category : ",max_cat)
s_hist_df['category_3'].fillna(max_cat,inplace = True)

Most frequently occuring category :  A


In [11]:
## replacing nan values in merchant_id with 'not known' category
s_hist_df['merchant_id'].fillna('not_known',inplace = True)

## replacing nan values in category_2 with 'other' category

max_cat = s_hist_df['category_2'].value_counts().index[0]
print("Most frequently occuring category : ",max_cat)
s_hist_df['category_2'].fillna(max_cat,inplace = True)

Most frequently occuring category :  1.0


In [12]:
s_hist_df.isnull().sum()*100/s_hist_df.shape[0]

authorized_flag         0.0
card_id                 0.0
city_id                 0.0
category_1              0.0
installments            0.0
category_3              0.0
merchant_category_id    0.0
merchant_id             0.0
month_lag               0.0
purchase_amount         0.0
purchase_date           0.0
category_2              0.0
state_id                0.0
subsector_id            0.0
new_purchase_amount     0.0
dtype: float64

## 2.2 Objective : Apply feature transformation on historical transactions
* So here we are going to apply some aggregation on categorical data grouped by card_id.
* The kind of aggregation to perform and the column to be aggregated is passed as an argument in computeCategoricalAggr function.
* Though we have observed in our EDA that 'mode' based aggregation doesn't correlate much with target variable.
* Still we are adding it to confirm our analysis in the end after feature selection.

In [13]:
import time

In [14]:
trans_hist_df = pd.DataFrame({'card_id' : s_hist_df['card_id'].unique()})

In [15]:
start_time = time.clock()
trans_hist_df = pd.merge(trans_hist_df,eda_fe.computeCategoricalAggr(s_hist_df,'category_1','mode'),on='card_id',how='inner')
end_time = time.clock()
print("time taken in seconds : ",end_time-start_time)

time taken in seconds :  75.4839920999998


In [16]:
start_time = time.clock()
trans_hist_df = pd.merge(trans_hist_df,eda_fe.computeCategoricalAggr(s_hist_df,'category_2','mode'),on='card_id',how='inner')
trans_hist_df = pd.merge(trans_hist_df,eda_fe.computeCategoricalAggr(s_hist_df,'category_3','mode'),on='card_id',how='inner')
trans_hist_df = pd.merge(trans_hist_df,eda_fe.computeCategoricalAggr(s_hist_df,'state_id','mode'),on='card_id',how='inner')
trans_hist_df = pd.merge(trans_hist_df,eda_fe.computeCategoricalAggr(s_hist_df,'city_id','mode'),on='card_id',how='inner')
end_time = time.clock()
print("time taken in seconds : ",end_time-start_time)

time taken in seconds :  234.56837479999967


In [17]:
## Nunique features
trans_hist_df = pd.merge(trans_hist_df,eda_fe.computeCategoricalAggr(s_hist_df,'category_1','nunique'),on='card_id',how='inner')
trans_hist_df = pd.merge(trans_hist_df,eda_fe.computeCategoricalAggr(s_hist_df,'category_2','nunique'),on='card_id',how='inner')
trans_hist_df = pd.merge(trans_hist_df,eda_fe.computeCategoricalAggr(s_hist_df,'category_3','nunique'),on='card_id',how='inner')
trans_hist_df = pd.merge(trans_hist_df,eda_fe.computeCategoricalAggr(s_hist_df,'state_id','nunique'),on='card_id',how='inner')
trans_hist_df = pd.merge(trans_hist_df,eda_fe.computeCategoricalAggr(s_hist_df,'city_id','nunique'),on='card_id',how='inner')

In [18]:
trans_hist_df.shape

(201917, 11)

In [19]:
trans_hist_df.head()

,card_id,category_1_mode,category_2_mode,category_3_mode,state_id_mode,city_id_mode,category_1_nunique,category_2_nunique,category_3_nunique,state_id_nunique,city_id_nunique
0,C_ID_5037ff576e,N,1.0,B,9,331,1,5,2,16,33
1,C_ID_0e171c1b48,N,4.0,A,13,277,1,3,2,3,4
2,C_ID_fc8e41b9cf,N,3.0,B,8,251,2,2,3,5,11
3,C_ID_b271e7ab60,N,4.0,A,22,17,1,2,2,3,3
4,C_ID_4bed29d75c,N,1.0,B,9,272,2,1,2,2,6


In [20]:
start_time = time.clock()
trans_hist_df = pd.merge(trans_hist_df,eda_fe.computeCategoricalAggr(s_hist_df,'subsector_id','mode'),on='card_id',how='inner')
trans_hist_df = pd.merge(trans_hist_df,eda_fe.computeCategoricalAggr(s_hist_df,'merchant_category_id','mode'),on='card_id',how='inner')

trans_hist_df = pd.merge(trans_hist_df,eda_fe.computeCategoricalAggr(s_hist_df,'subsector_id','nunique'),on='card_id',how='inner')
trans_hist_df = pd.merge(trans_hist_df,eda_fe.computeCategoricalAggr(s_hist_df,'merchant_category_id','nunique'),on='card_id',how='inner')

end_time = time.clock()
print("time taken in seconds : ",end_time-start_time)

time taken in seconds :  225.6875116000001


In [21]:
trans_hist_df.shape

(201917, 15)

In [22]:
## Intermediate saving of files
pickle.dump(trans_hist_df,open('trans_hist_df.pkl','wb'))

In [23]:
trans_hist_df = pickle.load(open('trans_hist_df.pkl','rb'))

### 2.2.3 Creating date-time related features

In [24]:
start_time = time.clock()
s_hist_df['purchase_date'] = pd.to_datetime(s_hist_df['purchase_date'])
s_hist_df['day'] = s_hist_df['purchase_date'].dt.day
s_hist_df['wday'] = s_hist_df['purchase_date'].dt.weekday
s_hist_df['month'] = s_hist_df['purchase_date'].dt.month
s_hist_df['year'] = s_hist_df['purchase_date'].dt.year
end_time = time.clock()
print("time taken in seconds : ",end_time-start_time)

time taken in seconds :  13.583130500000152


In [25]:
datetime_cols = ['day','wday','month','year']
stat_func = ['mode','min','count','nunique']

## For each of the datetime column we are going to apply 
## above aggregate functions and merge it with the trans_hist_df
## trans_hist_df is the dataframe containing features generated 
## using historical transactions

for col in tqdm(datetime_cols):
    for func in tqdm(stat_func): 
        trans_hist_df = pd.merge(trans_hist_df,eda_fe.computeCategoricalAggr(s_hist_df,col,func),on='card_id',how='inner')

100%|████████████████████████████████████████████████████████████████████████████████| 4/4 [1:19:42<00:00, 1195.74s/it]


In [26]:
trans_hist_df.head()

,card_id,category_1_mode,category_2_mode,category_3_mode,state_id_mode,city_id_mode,category_1_nunique,category_2_nunique,category_3_nunique,state_id_nunique,...,wday_count,wday_nunique,month_mode,month_min,month_count,month_nunique,year_mode,year_min,year_count,year_nunique
0,C_ID_5037ff576e,N,1.0,B,9,331,1,5,2,16,...,118,7,9,5,118,12,2017,2017,118,1
1,C_ID_0e171c1b48,N,4.0,A,13,277,1,3,2,3,...,279,7,2,11,279,12,2017,2018,279,2
2,C_ID_fc8e41b9cf,N,3.0,B,8,251,2,2,3,5,...,242,7,2,7,242,8,2017,2018,242,2
3,C_ID_b271e7ab60,N,4.0,A,22,17,1,2,2,3,...,85,7,2,4,85,12,2017,2018,85,2
4,C_ID_4bed29d75c,N,1.0,B,9,272,2,1,2,2,...,86,7,11,1,86,5,2017,2018,86,2


In [27]:
pickle.dump(trans_hist_df,open('trans_hist_df.pkl','wb'))

In [28]:
trans_hist_df = pickle.load(open('trans_hist_df.pkl','rb'))

In [29]:
s_hist_df.columns

Index(['authorized_flag', 'card_id', 'city_id', 'category_1', 'installments',
       'category_3', 'merchant_category_id', 'merchant_id', 'month_lag',
       'purchase_amount', 'purchase_date', 'category_2', 'state_id',
       'subsector_id', 'new_purchase_amount', 'day', 'wday', 'month', 'year'],
      dtype='object')

### Note
* Create date related boolean features.
* This adds up boolean features related to month_starts,month_ends,quarter_starts,quarter_ends,year_starts,year_ends and other important occasions.


In [ ]:
s_hist_df = pd.merge(s_hist_df,eda_fe.createDateRelatedBoolFeat(s_hist_df),on='card_id',how='inner')

In [32]:
pickle.dump(s_hist_df,open('s_hist_df.pkl','wb'))

In [4]:
s_hist_df = pickle.load(open('s_hist_df.pkl','rb'))

In [35]:
s_hist_df.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,...,wday,month,year,is_purchase_month_end,is_purchase_month_start,is_purchase_quarter_start,is_purchase_quarter_end,is_purchase_year_end,is_purchase_year_start,is_christmas
400,N,C_ID_5037ff576e,322,N,1,B,278,M_ID_b61c7d1be0,-3,-0.595260,...,3,9,2017,False,False,False,False,False,False,False
401,Y,C_ID_5037ff576e,138,N,1,B,307,M_ID_fe69229f24,-4,1.189469,...,0,8,2017,False,False,False,False,False,False,False
402,Y,C_ID_5037ff576e,138,N,1,B,705,M_ID_efc106141c,-9,-0.640069,...,6,3,2017,False,False,False,False,False,False,False
403,Y,C_ID_5037ff576e,226,N,1,B,307,M_ID_708022307c,-4,-0.652256,...,1,8,2017,False,False,False,False,False,False,False
404,Y,C_ID_5037ff576e,330,N,1,B,705,M_ID_393b4b8cec,-9,-0.674210,...,6,3,2017,False,False,False,False,False,False,False


In [36]:
# After Computing boolean datetime columns we are going to apply aggregate functions
# just like we did for day,weekday,month and year feature

# Columns to consider
bool_datetime_cols = [col for col in s_hist_df.columns if col.startswith('is_')]

# aggregate functions to apply on the above columns
stat_func = ['mode','min','count','nunique']

# loop to merge created features with generated features dataframe
for col in tqdm(bool_datetime_cols):
    for func in tqdm(stat_func): 
        trans_hist_df = pd.merge(trans_hist_df,eda_fe.computeCategoricalAggr(s_hist_df,col,func),on='card_id',how='inner')

100%|███████████████████████████████████████████████████████████████████████████████████| 7/7 [37:54<00:00, 324.98s/it]


In [37]:
trans_hist_df.head()

,card_id,category_1_mode,category_2_mode,category_3_mode,state_id_mode,city_id_mode,category_1_nunique,category_2_nunique,category_3_nunique,state_id_nunique,...,is_purchase_year_end_count,is_purchase_year_end_nunique,is_purchase_year_start_mode,is_purchase_year_start_min,is_purchase_year_start_count,is_purchase_year_start_nunique,is_christmas_mode,is_christmas_min,is_christmas_count,is_christmas_nunique
0,C_ID_5037ff576e,N,1.0,B,9,331,1,5,2,16,...,118,1,False,False,118,1,False,True,118,2
1,C_ID_0e171c1b48,N,4.0,A,13,277,1,3,2,3,...,279,1,False,True,279,2,False,False,279,1
2,C_ID_fc8e41b9cf,N,3.0,B,8,251,2,2,3,5,...,242,1,False,False,242,1,False,False,242,1
3,C_ID_b271e7ab60,N,4.0,A,22,17,1,2,2,3,...,85,1,False,False,85,1,False,False,85,1
4,C_ID_4bed29d75c,N,1.0,B,9,272,2,1,2,2,...,86,2,False,True,86,2,False,False,86,1


In [38]:
# Storing dataframe for further use
pickle.dump(trans_hist_df,open('trans_hist_df.pkl','wb'))

In [39]:
# Reading the dataframe
trans_hist_df = pickle.load(open('trans_hist_df.pkl','rb'))

## 2.2.4 Create purchase amount features

In [2]:
## Aggregate functions
aggr_funcs = ['min','max','mean','sum','std']

In [41]:
#Creating dataframe for data grouped by card_id and month_lag,aggregating over new_purchase_amounts
purch_lag_df = eda_fe.createPurchaseAmountLagFeatures(s_hist_df,aggr_funcs)

Creating purchase_amount aggregation wrt month_lags...


 14%|███████████▊                                                                       | 2/14 [00:00<00:00, 15.12it/s]

Merging aggregate columns...



100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:24<00:00,  4.81s/it]


In [42]:
purch_lag_df.shape

(201917, 76)

In [43]:
purch_lag_df.isnull().sum()*100/purch_lag_df.shape[0]

card_id              0.000000
np_amount_min_12    75.010524
np_amount_min_11    70.324440
np_amount_min_10    66.350035
np_amount_min_9     61.776869
                      ...    
np_amount_std_2     18.545244
np_amount_std_1     19.164805
np_amount_std_0     19.627867
month_lag           10.008073
np_amount_std_13    82.238742
Length: 76, dtype: float64

* A lot of values are null because not all card_id's have transactions related to all month lags. 

### 2.2.4.1 Creating purchase amount ratio features with lag shift = 2

* purchase_ratio_df is a dataframe containing ratios of purchase amounts in different month lags.
* ratio = purch_lag_df['future month_lag']/purch_lag_df['prev_month_lag']
* These purchase_ratios are calculated over a shift of 2.
* reference date : 02-01-2018(mm-dd-yyyy)
* Here a lag of 0 means the current month wrt to reference date
* A lag of 2(i.e -2 have converted it to positive values for understandability purpose) means 2 months behind the reference date.
* np_amount_min_0_2 = purch_lag_df['np_amount_min_0']/purch_lag_df['np_amount_min_2']
* Similarly the other purchase amt ratio columns are as follows: np_amount_min_1_3,np_amount_min_2_4,np_amount_min_3_5


In [44]:
purch_ratio_df = eda_fe.createPurchaseAmountRatioFeatures(purch_lag_df,aggr_funcs,2)

100%|█████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 171.88it/s]

100%|█████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 267.38it/s]

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 14.88it/s]


In [45]:
null_df = ((purch_ratio_df.isnull().sum()*100/purch_ratio_df.shape[0]) > 50).reset_index().rename(columns = {0 : 'bool'})
len(null_df[null_df['bool'] == True]['index'])

37

* About 37 columns from the purchase_amount_ratio's are empty.
* So we'll remove them later in the feature selection stage.
* For now we'll just merge the dataframes trans_hist_df and purchase_amount_ratios.

In [46]:
trans_hist_df = pd.merge(trans_hist_df,purch_ratio_df,on='card_id',how='inner')

In [47]:
trans_hist_df.shape

(201917, 119)

### 2.2.4.2 Let's also merge the purchase_lag_df

In [48]:
trans_hist_df = pd.merge(trans_hist_df,purch_lag_df,on='card_id',how='inner')
trans_hist_df.shape

(201917, 194)

### 2.2.4.3 Let's create purchase_amount_ratio_features with lag shift = 1

In [49]:

purch_ratio_lag1_df = eda_fe.createPurchaseAmountRatioFeatures(purch_lag_df,aggr_funcs,1)

100%|█████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 160.36it/s]

100%|█████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 240.65it/s]

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 17.78it/s]


In [50]:
trans_hist_df = pd.merge(trans_hist_df,purch_ratio_lag1_df,on='card_id',how='inner')

In [51]:
trans_hist_df.shape

(201917, 254)

### 2.2.4.4 Let's create simple window averaging features
* Simple window averages are averaging purch_lag_df columns over a period of specified window.
* In below cells we have considered windows of size 2 and 3
* So the features which will be formed will be as follows:
* np_amount_min_0_2_avg = np_amount_min(average over a month lag of 0-2)
* np_amount_min_1_3_avg = np_amount_min(average over a month lag of 1-3)

In [52]:
purch_wavg_lag2_df = eda_fe.createPurchaseAmountWindowAvg(purch_lag_df,aggr_funcs,2)
purch_wavg_lag3_df = eda_fe.createPurchaseAmountWindowAvg(purch_lag_df,aggr_funcs,3)

In [53]:
trans_hist_df = pd.merge(trans_hist_df,purch_wavg_lag2_df,on='card_id',how='inner')
trans_hist_df = pd.merge(trans_hist_df,purch_wavg_lag3_df,on='card_id',how='inner')
trans_hist_df.shape

(201917, 369)

## 2.2.5 Let's create installments related features
* Applying categorical aggregation 
* Applying numerical aggregation with groupby on card_id,installments and aggregation over new_purchase_amount

In [54]:
trans_hist_df = pd.merge(trans_hist_df,eda_fe.computeCategoricalAggr(s_hist_df,'installments','mode'),on='card_id',how='inner')
trans_hist_df = pd.merge(trans_hist_df,eda_fe.computeCategoricalAggr(s_hist_df,'installments','nunique'),on='card_id',how='inner')
trans_hist_df.shape

(201917, 371)

In [55]:
pickle.dump(trans_hist_df,open('trans_hist_df.pkl','wb'))

In [56]:
trans_hist_df = pd.merge(trans_hist_df,eda_fe.computeCategoricalAggr(s_hist_df,'installments','count'),on='card_id',how='inner')
trans_hist_df = pd.merge(trans_hist_df,eda_fe.computeCategoricalAggr(s_hist_df,'installments','min'),on='card_id',how='inner')

In [57]:
pickle.dump(trans_hist_df,open('trans_hist_df.pkl','wb'))

In [5]:
trans_hist_df = pickle.load(open('trans_hist_df.pkl','rb'))
trans_hist_df.head()

,card_id,category_1_mode,category_2_mode,category_3_mode,state_id_mode,city_id_mode,category_1_nunique,category_2_nunique,category_3_nunique,state_id_nunique,...,np_amount_std_5_8_avg,np_amount_std_6_9_avg,np_amount_std_7_10_avg,np_amount_std_8_11_avg,np_amount_std_9_12_avg,np_amount_std_10_13_avg,installments_mode,installments_nunique,installments_count,installments_min
0,C_ID_5037ff576e,N,1.0,B,9,331,1,5,2,16,...,0.0,0.0,0.0,0.0,0.0,0.0,1,3,118,3
1,C_ID_0e171c1b48,N,4.0,A,13,277,1,3,2,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0,2,279,1
2,C_ID_fc8e41b9cf,N,3.0,B,8,251,2,2,3,5,...,0.0,0.0,0.0,0.0,0.0,0.0,1,8,242,6
3,C_ID_b271e7ab60,N,4.0,A,22,17,1,2,2,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0,2,85,1
4,C_ID_4bed29d75c,N,1.0,B,9,272,2,1,2,2,...,0.0,0.0,0.0,0.0,0.0,0.0,1,4,86,2


In [6]:
trans_hist_df.shape

(201917, 373)

In [ ]:
## Applying numerical aggregation 
temp_df = generateAggrColumns(s_hist_df,'installments','new_purchase_amount',aggr_funcs,'install',True)
temp_df.shape

In [27]:
temp_df.columns

Index(['card_id', 'install_1_min', 'install_1_max', 'install_1_mean',
       'install_1_sum', 'install_1_std', 'install_5_min', 'install_5_max',
       'install_5_mean', 'install_5_sum', 'install_5_std', 'install_3_min',
       'install_3_max', 'install_3_mean', 'install_3_sum', 'install_3_std',
       'install_0_min', 'install_0_max', 'install_0_mean', 'install_0_sum',
       'install_0_std', 'install_-1_min', 'install_-1_max', 'install_-1_mean',
       'install_-1_sum', 'install_-1_std', 'install_4_min', 'install_4_max',
       'install_4_mean', 'install_4_sum', 'install_4_std', 'install_2_min',
       'install_2_max', 'install_2_mean', 'install_2_sum', 'install_2_std',
       'install_10_min', 'install_10_max', 'install_10_mean', 'install_10_sum',
       'install_10_std', 'install_6_min', 'install_6_max', 'install_6_mean',
       'install_6_sum', 'install_6_std', 'install_7_min', 'install_7_max',
       'install_7_mean', 'install_7_sum', 'install_7_std', 'install_12_min',
       'in

In [28]:
trans_hist_df = pd.merge(trans_hist_df,temp_df,on='card_id',how='left')
trans_hist_df.shape

(201917, 448)

In [29]:
pickle.dump(trans_hist_df,open('trans_hist_df.pkl','wb'))

## 2.2.6 Let's create TFIDF-Vectorizer and  word2vec Vectorizer
* Creating tfidf and word2vec features for columns merchant_category_id and subsector_id groupedby card_id.
* We group the dataframe by card_id's and merchant_category_id,create a list of all merchant category_ids for each card_id.
* Apply tfidf and word2Vec transformation on it 
* Reduce the number of dimensions by svd in case of tfidf.

In [40]:
## The function returns a tuple with svd columns and svd object to later transform the test data
svd_cat_id_df = eda_fe.createSvdFeatures(s_hist_df,'card_id','merchant_category_id',40)
svd_subsector_id_df = eda_fe.createSvdFeatures(s_hist_df,'card_id','subsector_id',10)

Performing tfidf vectorization on  merchant_category_id ...
Explained Variance ration with  40  components :  72.73859925096701
Time taken for completion :  -55.41503190000003
Performing tfidf vectorization on  subsector_id ...
Explained Variance ration with  10  components :  78.530972541119
Time taken for completion :  -66.19161429999986


In [58]:
## Even here the first object is a dataframe containing columns of word2Vec and second object is a word2Vec transformer
w2v_cat_id_df = eda_fe.createWord2VecFeatures(s_hist_df,'card_id','merchant_category_id',30)
w2v_ssector_id_df = eda_fe.createWord2VecFeatures(s_hist_df,'card_id','subsector_id',10)

Time taken for completion :  -110.80932740000026
Time taken for completion :  -83.7628249999998


In [42]:
pickle.dump(svd_cat_id_df[1],open('merch_cat_svd_object','wb'))
pickle.dump(svd_subsector_id_df[1],open('subsector_svd_object','wb'))

In [43]:
pickle.dump(w2v_cat_id_df[1],open('merch_cat_w2v_object','wb'))
pickle.dump(w2v_ssector_id_df[1],open('subsector_w2v_object','wb'))

In [44]:

svd_cat_id_df = svd_cat_id_df[0]
svd_subsector_id_df=svd_subsector_id_df[0]

In [59]:
w2v_cat_id_df = w2v_cat_id_df[0]
w2v_ssector_id_df = w2v_ssector_id_df[0]

In [46]:
merch_cat_cols = ['merch_cat_' + col for col in svd_cat_id_df.columns if col.startswith('svd')]
print("Merchant Category Id SVD Columns : ",merch_cat_cols)
subsector_cols = ['ssector_' + col for col in svd_subsector_id_df.columns if col.startswith('svd')]
print("Subsector SVD Columns : ",subsector_cols)

Merchant Category Id SVD Columns :  ['merch_cat_svd_0', 'merch_cat_svd_1', 'merch_cat_svd_2', 'merch_cat_svd_3', 'merch_cat_svd_4', 'merch_cat_svd_5', 'merch_cat_svd_6', 'merch_cat_svd_7', 'merch_cat_svd_8', 'merch_cat_svd_9', 'merch_cat_svd_10', 'merch_cat_svd_11', 'merch_cat_svd_12', 'merch_cat_svd_13', 'merch_cat_svd_14', 'merch_cat_svd_15', 'merch_cat_svd_16', 'merch_cat_svd_17', 'merch_cat_svd_18', 'merch_cat_svd_19', 'merch_cat_svd_20', 'merch_cat_svd_21', 'merch_cat_svd_22', 'merch_cat_svd_23', 'merch_cat_svd_24', 'merch_cat_svd_25', 'merch_cat_svd_26', 'merch_cat_svd_27', 'merch_cat_svd_28', 'merch_cat_svd_29', 'merch_cat_svd_30', 'merch_cat_svd_31', 'merch_cat_svd_32', 'merch_cat_svd_33', 'merch_cat_svd_34', 'merch_cat_svd_35', 'merch_cat_svd_36', 'merch_cat_svd_37', 'merch_cat_svd_38', 'merch_cat_svd_39']
Subsector SVD Columns :  ['ssector_svd_0', 'ssector_svd_1', 'ssector_svd_2', 'ssector_svd_3', 'ssector_svd_4', 'ssector_svd_5', 'ssector_svd_6', 'ssector_svd_7', 'ssector_sv

In [49]:
svd_cat_id_df.drop(['list'],axis = 1,inplace = True)
svd_subsector_id_df.drop(['list'],axis = 1,inplace = True)

In [50]:
svd_cat_id_df.columns = ['card_id'] + merch_cat_cols
svd_subsector_id_df.columns = ['card_id'] + subsector_cols

In [51]:
svd_cat_id_df.shape

(201917, 41)

In [52]:
trans_hist_df = pd.merge(trans_hist_df,svd_cat_id_df,on='card_id',how='inner')
trans_hist_df = pd.merge(trans_hist_df,svd_subsector_id_df,on='card_id',how='inner')
trans_hist_df.shape

(201917, 498)

In [60]:
w2v_merch_cat_cols = ['merch_cat_' + col for col in w2v_cat_id_df.columns if col.startswith('w2v')]
print("Merchant Category Id w2v Columns : ",w2v_merch_cat_cols)
w2v_subsector_cols = ['ssector_' + col for col in w2v_ssector_id_df.columns if col.startswith('w2v')]
print("Subsector W2V Columns : ",w2v_subsector_cols)

w2v_cat_id_df.columns = ['card_id'] + w2v_merch_cat_cols
w2v_ssector_id_df.columns = ['card_id'] + w2v_subsector_cols

Merchant Category Id w2v Columns :  ['merch_cat_w2v_0', 'merch_cat_w2v_1', 'merch_cat_w2v_2', 'merch_cat_w2v_3', 'merch_cat_w2v_4', 'merch_cat_w2v_5', 'merch_cat_w2v_6', 'merch_cat_w2v_7', 'merch_cat_w2v_8', 'merch_cat_w2v_9', 'merch_cat_w2v_10', 'merch_cat_w2v_11', 'merch_cat_w2v_12', 'merch_cat_w2v_13', 'merch_cat_w2v_14', 'merch_cat_w2v_15', 'merch_cat_w2v_16', 'merch_cat_w2v_17', 'merch_cat_w2v_18', 'merch_cat_w2v_19', 'merch_cat_w2v_20', 'merch_cat_w2v_21', 'merch_cat_w2v_22', 'merch_cat_w2v_23', 'merch_cat_w2v_24', 'merch_cat_w2v_25', 'merch_cat_w2v_26', 'merch_cat_w2v_27', 'merch_cat_w2v_28', 'merch_cat_w2v_29']
Subsector W2V Columns :  ['ssector_w2v_0', 'ssector_w2v_1', 'ssector_w2v_2', 'ssector_w2v_3', 'ssector_w2v_4', 'ssector_w2v_5', 'ssector_w2v_6', 'ssector_w2v_7', 'ssector_w2v_8', 'ssector_w2v_9']


In [63]:
trans_hist_df = pd.merge(trans_hist_df,w2v_cat_id_df,on='card_id',how='inner')
trans_hist_df = pd.merge(trans_hist_df,w2v_ssector_id_df,on='card_id',how='inner')
trans_hist_df.shape

(201917, 538)

In [64]:
pickle.dump(trans_hist_df,open('trans_hist_df.pkl','wb'))

## 3. Objective Create Aggregation for merchants data

In [5]:
## Creating dataframe of subset of columns
## Don't want to use all the columns just a few of them 
## for the purpose of joining merchants data with historical transactions
hist_card_merch_df = s_hist_df[['card_id','merchant_id']]

In [6]:
merchants_df = pd.read_csv('merchants.csv')

In [7]:
merchants_df.head()

,merchant_id,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,category_1,most_recent_sales_range,most_recent_purchases_range,avg_sales_lag3,...,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,city_id,state_id,category_2
0,M_ID_838061e48c,8353,792,9,-0.057471,-0.057471,N,E,E,-0.40,...,-2.25,18.666667,6,-2.32,13.916667,12,N,242,9,1.0
1,M_ID_9339d880ad,3184,840,20,-0.057471,-0.057471,N,E,E,-0.72,...,-0.74,1.291667,6,-0.57,1.687500,12,N,22,16,1.0
2,M_ID_e726bbae1e,447,690,1,-0.057471,-0.057471,N,E,E,-82.13,...,-82.13,260.000000,2,-82.13,260.000000,2,N,-1,5,5.0
3,M_ID_a70e9c5f81,5026,792,9,-0.057471,-0.057471,Y,E,E,NaN,...,NaN,4.666667,6,NaN,3.833333,12,Y,-1,-1,NaN
4,M_ID_64456c37ce,2228,222,21,-0.057471,-0.057471,Y,E,E,NaN,...,NaN,0.361111,6,NaN,0.347222,12,Y,-1,-1,NaN


In [8]:
hist_card_merch_df = pd.merge(hist_card_merch_df,merchants_df,on='merchant_id',how='inner')
hist_card_merch_df.shape

(18700121, 23)

In [9]:
pickle.dump(hist_card_merch_df,open('hist_card_merch_df','wb'))

In [12]:
trans_merch_df = pd.DataFrame({'card_id' : hist_card_merch_df['card_id'].unique()})
trans_merch_df.shape

(201917, 1)

## 3.1 Objective : Applying categorical transformations and aggregations

In [13]:
columns = ['category_4','most_recent_sales_range','most_recent_purchases_range']
for col in tqdm(columns):
    trans_merch_df = pd.merge(trans_merch_df,eda_fe.computeCategoricalAggr(hist_card_merch_df,col,'mode'),on='card_id',how='inner')
    trans_merch_df = pd.merge(trans_merch_df,eda_fe.computeCategoricalAggr(hist_card_merch_df,col,'nunique'),on='card_id',how='inner')
trans_merch_df.shape

100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [12:38<00:00, 252.83s/it]


(201917, 7)

In [14]:
trans_merch_df.head()

,card_id,category_4_mode,category_4_nunique,most_recent_sales_range_mode,most_recent_sales_range_nunique,most_recent_purchases_range_mode,most_recent_purchases_range_nunique
0,C_ID_5037ff576e,N,2,B,5,C,5
1,C_ID_ae1363307b,Y,2,C,5,C,5
2,C_ID_1004bb5144,N,2,C,5,C,5
3,C_ID_ea09f86dad,N,2,C,5,C,5
4,C_ID_24f394f987,N,2,D,5,D,5


## 3.2 Objective : Applying numerical aggregations

In [18]:
columns = ['numerical_1','numerical_2','avg_sales_lag3','avg_sales_lag6','avg_sales_lag12',
          'avg_purchases_lag3','avg_purchases_lag6','avg_purchases_lag12']

for col in columns:
    trans_merch_df = pd.merge(trans_merch_df,eda_fe.createNumericalAggr(hist_card_merch_df,'card_id',col),on='card_id',how='inner')
trans_merch_df.shape

(201917, 47)

## 3.3 Objective :Applying categorical aggregations

In [ ]:
columns = ['active_months_lag3','active_months_lag6','active_months_lag12']
for col in tqdm(columns):
    trans_merch_df = pd.merge(trans_merch_df,eda_fe.computeCategoricalAggr(hist_card_merch_df,col,'mode'),on='card_id',how='inner')
    trans_merch_df = pd.merge(trans_merch_df,eda_fe.computeCategoricalAggr(hist_card_merch_df,col,'nunique'),on='card_id',how='inner')
trans_merch_df.shape

 67%|███████████████████████████████████████████████████████▎                           | 2/3 [07:09<03:43, 223.25s/it]

In [ ]:
pickle.dump(trans_merch_df,'trans_merch_df.pkl')

## 4. Apply transformations on new merchant transactions data
* Transformations to be applied for new_merchant transactions are similar to that of historical transactions

In [4]:
new_trans_df = eda_fe.reduce_mem_usage(pd.read_csv('new_merchant_transactions.csv'))
new_trans_df['purchase_amount_new'] = np.round(new_trans_df['purchase_amount'] / 0.00150265118 + 497.06,2)

Mem. usage decreased to 114.20 Mb (45.5% reduction)


c:\users\capiot\appdata\local\programs\python\python36\lib\site-packages\pandas\core\series.py:2145: RuntimeWarning: overflow encountered in multiply
  result = com.values_from_object(self).round(decimals)


In [5]:
train_card_ids = pickle.load(open('train_card_ids.pkl','rb'))

In [6]:
s_new_df = new_trans_df.loc[new_trans_df['card_id'].isin(train_card_ids)].copy()
s_new_df.shape

(1219685, 15)

In [7]:
print("Percentage of data sampled based on card_id from train data : ",s_new_df.shape[0]*100/new_trans_df.shape[0])

Percentage of data sampled based on card_id from train data :  62.13274268210741


In [8]:
s_new_df.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,purchase_amount_new
4,Y,C_ID_ef55cf8d4b,-1,Y,1,B,166,M_ID_3c86fa3831,1,-0.659668,2018-03-22 21:07:53,NaN,-1,29,58.0
5,Y,C_ID_ef55cf8d4b,231,N,1,B,367,M_ID_8874615e00,2,-0.632812,2018-04-02 12:53:28,1.0,9,16,76.0
6,Y,C_ID_ef55cf8d4b,69,N,1,B,333,M_ID_6d061b5ddc,1,5.265625,2018-03-28 19:50:19,1.0,9,10,inf
7,Y,C_ID_ef55cf8d4b,231,N,1,B,307,M_ID_df1e022f41,2,-0.553711,2018-04-05 08:06:52,1.0,9,19,128.5
8,Y,C_ID_ef55cf8d4b,69,N,1,B,278,M_ID_d15eae0468,2,-0.596680,2018-04-07 18:37:40,1.0,9,37,100.0


In [9]:
del(new_trans_df)

In [10]:
s_new_df.rename(columns = {'purchase_amount_new' : 'new_purchase_amount'},inplace = True)
s_new_df.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,new_purchase_amount
4,Y,C_ID_ef55cf8d4b,-1,Y,1,B,166,M_ID_3c86fa3831,1,-0.659668,2018-03-22 21:07:53,NaN,-1,29,58.0
5,Y,C_ID_ef55cf8d4b,231,N,1,B,367,M_ID_8874615e00,2,-0.632812,2018-04-02 12:53:28,1.0,9,16,76.0
6,Y,C_ID_ef55cf8d4b,69,N,1,B,333,M_ID_6d061b5ddc,1,5.265625,2018-03-28 19:50:19,1.0,9,10,inf
7,Y,C_ID_ef55cf8d4b,231,N,1,B,307,M_ID_df1e022f41,2,-0.553711,2018-04-05 08:06:52,1.0,9,19,128.5
8,Y,C_ID_ef55cf8d4b,69,N,1,B,278,M_ID_d15eae0468,2,-0.596680,2018-04-07 18:37:40,1.0,9,37,100.0


In [11]:
pickle.dump(s_new_df,open('s_new_df.pkl','wb'))

In [10]:
trans_new_df = pd.DataFrame({'card_id' : s_new_df['card_id'].unique()})
trans_new_df.shape

(179986, 1)

## 4.1 Applying mode and nunique features on new_transactions_data

In [12]:
columns = ['category_1','category_2','category_3','city_id','state_id','subsector_id','merchant_category_id']
for col in tqdm(columns):
    trans_new_df = pd.merge(trans_new_df,eda_fe.computeCategoricalAggr(s_new_df,col,'mode'),on='card_id',how='inner')
    trans_new_df = pd.merge(trans_new_df,eda_fe.computeCategoricalAggr(s_new_df,col,'nunique'),on='card_id',how='inner')

trans_new_df.shape

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [10:45<00:00, 92.19s/it]


(179986, 16)

In [15]:
trans_new_df.drop(['category_1_mode_x'],axis=1,inplace = True)
trans_new_df.columns

Index(['card_id', 'category_1_mode_y', 'category_1_nunique', 'category_2_mode',
       'category_2_nunique', 'category_3_mode', 'category_3_nunique',
       'city_id_mode', 'city_id_nunique', 'state_id_mode', 'state_id_nunique',
       'subsector_id_mode', 'subsector_id_nunique',
       'merchant_category_id_mode', 'merchant_category_id_nunique'],
      dtype='object')

In [16]:
trans_new_df.rename(columns={'category_1_mode_y' : 'category_1_mode'},inplace=True)

In [18]:
pickle.dump(trans_new_df,open('trans_new_df','wb'))

## 4.2 Applying date time features

In [17]:
s_new_df['purchase_date'] = pd.to_datetime(s_new_df['purchase_date'])
s_new_df['day'] = s_new_df['purchase_date'].dt.day
s_new_df['wday'] = s_new_df['purchase_date'].dt.weekday
s_new_df['month'] = s_new_df['purchase_date'].dt.month
s_new_df['year'] = s_new_df['purchase_date'].dt.year

In [22]:
datetime_cols = ['day','wday','month','year']
stat_func = ['mode','min','count','nunique']
for col in tqdm(datetime_cols):
    for func in tqdm(stat_func): 
        trans_new_df = pd.merge(trans_new_df,eda_fe.computeCategoricalAggr(s_new_df,col,func),on='card_id',how='inner')

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [19:25<00:00, 291.43s/it]


In [24]:
trans_new_df.shape

(179986, 31)

In [27]:
pickle.dump(trans_new_df,open('trans_new_df.pkl','wb'))

## 4.3 Applying purchase lag transformations

In [12]:
trans_new_df = pickle.load(open('trans_new_df.pkl','rb'))

In [13]:
purchase_lag_df = eda_fe.createPurchaseAmountLagFeatures(s_new_df,aggr_funcs)
purchase_lag_df.head()

Creating purchase_amount aggregation wrt month_lags...


  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

Merging aggregate columns...



100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:03<00:00,  1.58it/s]


,card_id,np_amount_min_2,np_amount_min_1,np_amount_max_2,np_amount_max_1,np_amount_mean_2,np_amount_mean_1,np_amount_sum_2,np_amount_sum_1,np_amount_std_2,np_amount_std_1
0,C_ID_00007093c1,50.0,NaN,60.000,NaN,55.00000,NaN,110.000,NaN,7.070312,NaN
1,C_ID_000183fdda,36.0,9.750000,425.500,191.25,168.87500,71.9375,506.500,575.5,222.375000,64.1250
2,C_ID_0002c7c2c1,10.0,22.234375,68.750,165.75,42.90625,79.5000,128.750,238.5,30.015625,76.0000
3,C_ID_0003be3c83,8.0,225.500000,35.000,385.25,20.31250,305.5000,81.250,611.0,11.109375,112.9375
4,C_ID_00042d509c,17.5,NaN,185.875,NaN,101.68750,NaN,203.375,NaN,119.062500,NaN


In [15]:
purchase_lag_df.shape

(179986, 11)

In [42]:
trans_new_df = pd.merge(trans_new_df,purch_lag_df,on = 'card_id',how='inner')

In [43]:
trans_new_df.shape

(179986, 41)

In [60]:
purch_lag_df.head()

,card_id,np_amount_min_2,np_amount_min_1,np_amount_max_2,np_amount_max_1,np_amount_mean_2,np_amount_mean_1,np_amount_sum_2,np_amount_sum_1,np_amount_std_2,np_amount_std_1
0,C_ID_00007093c1,50.0,NaN,60.000,NaN,55.00000,NaN,110.000,NaN,7.070312,NaN
1,C_ID_000183fdda,36.0,9.750000,425.500,191.25,168.87500,71.9375,506.500,575.5,222.375000,64.1250
2,C_ID_0002c7c2c1,10.0,22.234375,68.750,165.75,42.90625,79.5000,128.750,238.5,30.015625,76.0000
3,C_ID_0003be3c83,8.0,225.500000,35.000,385.25,20.31250,305.5000,81.250,611.0,11.109375,112.9375
4,C_ID_00042d509c,17.5,NaN,185.875,NaN,101.68750,NaN,203.375,NaN,119.062500,NaN


In [75]:
month_lag = s_new_df['month_lag'].unique()
purch_amt_ratio_df = eda_fe.createPurchaseAmountRatioFeatures(purch_lag_df,aggr_funcs,1,month_lag)
purch_amt_ratio_df.shape

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 46.30it/s]

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 251.64it/s]

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 180.09it/s]

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 220.67it/s]

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 81.15it/s]


(179986, 6)

In [76]:
purch_amt_ratio_df.head()

,np_amount_min_1_2,np_amount_max_1_2,np_amount_mean_1_2,np_amount_sum_1_2,np_amount_std_1_2,card_id
0,NaN,NaN,NaN,NaN,NaN,C_ID_00007093c1
1,3.691406,2.224609,2.347656,0.879883,3.468750,C_ID_000183fdda
2,0.449707,0.414795,0.539551,0.540039,0.395020,C_ID_0002c7c2c1
3,0.035492,0.090820,0.066467,0.132935,0.098389,C_ID_0003be3c83
4,NaN,NaN,NaN,NaN,NaN,C_ID_00042d509c


In [78]:
trans_new_df = pd.merge(trans_new_df,purch_amt_ratio_df,on='card_id',how='inner')
trans_new_df.shape

(179986, 46)

In [79]:
pickle.dump(trans_new_df,open('trans_new_df.pkl','wb'))

## 4.4 Applying simple window averaging

In [16]:
month_lag = s_new_df['month_lag'].unique()
purch_avg_df = eda_fe.createPurchaseAmountWindowAvg(purchase_lag_df,aggr_funcs,1,month_lag)
purch_avg_df.head()

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 51.26it/s]

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 160.05it/s]

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 132.93it/s]

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 215.97it/s]

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 65.93it/s]


,np_amount_min_1_2_avg,np_amount_max_1_2_avg,np_amount_mean_1_2_avg,np_amount_sum_1_2_avg,np_amount_std_1_2_avg,card_id
0,NaN,NaN,NaN,NaN,NaN,C_ID_00007093c1
1,4.875000,95.625,35.96875,287.75,32.06250,C_ID_000183fdda
2,11.117188,82.875,39.75000,119.25,38.00000,C_ID_0002c7c2c1
3,112.750000,192.625,152.75000,305.50,56.46875,C_ID_0003be3c83
4,NaN,NaN,NaN,NaN,NaN,C_ID_00042d509c


In [17]:
purch_avg_df.shape

(179986, 6)

In [18]:
trans_new_df = pd.merge(trans_new_df,purch_avg_df,on='card_id',how='inner')
trans_new_df.shape

(179986, 51)

In [19]:
pickle.dump(trans_new_df,open('trans_new_df.pkl','wb'))

In [20]:
trans_new_df.columns

Index(['card_id', 'category_1_mode', 'category_1_nunique', 'category_2_mode',
       'category_2_nunique', 'category_3_mode', 'category_3_nunique',
       'city_id_mode', 'city_id_nunique', 'state_id_mode', 'state_id_nunique',
       'subsector_id_mode', 'subsector_id_nunique',
       'merchant_category_id_mode', 'merchant_category_id_nunique', 'day_mode',
       'day_min', 'day_count', 'day_nunique', 'wday_mode', 'wday_min',
       'wday_count', 'wday_nunique', 'month_mode', 'month_min', 'month_count',
       'month_nunique', 'year_mode', 'year_min', 'year_count', 'year_nunique',
       'np_amount_min_2', 'np_amount_min_1', 'np_amount_max_2',
       'np_amount_max_1', 'np_amount_mean_2', 'np_amount_mean_1',
       'np_amount_sum_2', 'np_amount_sum_1', 'np_amount_std_2',
       'np_amount_std_1', 'np_amount_min_1_2', 'np_amount_max_1_2',
       'np_amount_mean_1_2', 'np_amount_sum_1_2', 'np_amount_std_1_2',
       'np_amount_min_1_2_avg', 'np_amount_max_1_2_avg',
       'np_amount_mean

## 4.5 Creating installment related features

In [32]:
aggr_funcs = ['mode','nunique','min','count']

In [21]:
for func in tqdm(aggr_funcs):
    trans_new_df = pd.merge(trans_new_df,eda_fe.computeCategoricalAggr(s_new_df,'installments',func),on='card_id',how='inner')
trans_new_df.shape

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [08:55<00:00, 133.76s/it]


(179986, 55)

In [23]:
pickle.dump(trans_new_df,open('trans_new_df.pkl','wb'))

In [3]:
s_new_df = pickle.load(open('s_new_df.pkl','rb'))
trans_new_df = pickle.load(open('trans_new_df.pkl','rb'))

In [4]:
temp_df = eda_fe.generateAggrColumns(s_new_df,'installments','new_purchase_amount',aggr_funcs,'install',True)

  0%|                                                                                           | 0/15 [00:00<?, ?it/s]

Category of Column :  1


  7%|█████▌                                                                             | 1/15 [00:00<00:09,  1.44it/s]

Category of Column :  -1


 13%|███████████                                                                        | 2/15 [00:00<00:07,  1.77it/s]

...

***********Merge_df**********
(179982, 66)
Category of Column :  11


 93%|████████████████████████████████████████████████████████████████████████████▌     | 14/15 [00:05<00:00,  2.29it/s]

***********Merge_df**********
(179986, 71)
Category of Column :  999


100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:06<00:00,  2.45it/s]

***********Merge_df**********
(179986, 76)


In [6]:
trans_new_df = pd.merge(trans_new_df,temp_df,on='card_id',how='inner')

In [8]:
pickle.dump(trans_new_df,open('trans_new_df.pkl','wb'))

## 4.6 Creating encoded features for merchant_category_id and subsector_id

In [14]:
svd_cat_id_df,new_svd_merch_cat_obj = eda_fe.createSvdFeatures(s_new_df,'card_id','merchant_category_id',40)
svd_ssector_id_df,new_svd_ssector_obj = eda_fe.createSvdFeatures(s_new_df,'card_id','subsector_id',10)

Performing tfidf vectorization on  merchant_category_id ...
Explained Variance ration with  40  components :  65.78329288138652
Time taken for completion :  -54.11445480000003
Performing tfidf vectorization on  subsector_id ...
Explained Variance ration with  10  components :  70.83104713505453
Time taken for completion :  -47.477130500000044


In [15]:
pickle.dump(new_svd_merch_cat_obj,open('new_svd_merchcat_obj.pkl','wb'))
pickle.dump(new_svd_ssector_obj,open('new_svd_ssector_obj.pkl','wb'))

In [16]:
svd_cat_id_df.drop(['list'],axis=1,inplace=True)
svd_ssector_id_df.drop(['list'],axis=1,inplace=True)

In [18]:
trans_new_df = pd.merge(trans_new_df,svd_cat_id_df,on='card_id',how='inner')
trans_new_df = pd.merge(trans_new_df,svd_ssector_id_df,on='card_id',how='inner')
trans_new_df.shape

(179986, 180)

In [20]:
w2v_cat_id_df,new_w2v_merch_cat_obj = eda_fe.createWord2VecFeatures(s_new_df,'card_id','merchant_category_id',40)
w2v_ssector_id_df,new_w2v_ssector_obj = eda_fe.createWord2VecFeatures(s_new_df,'card_id','subsector_id',10)

Time taken for completion :  -64.70854950000012
Time taken for completion :  -59.57170659999997


In [21]:
pickle.dump(new_w2v_merch_cat_obj,open('new_w2v_merchcat_obj.pkl','wb'))
pickle.dump(new_w2v_ssector_obj,open('new_w2v_ssector_obj.pkl','wb'))

In [24]:
trans_new_df = pd.merge(trans_new_df,w2v_cat_id_df,on='card_id',how='inner')
trans_new_df = pd.merge(trans_new_df,w2v_ssector_id_df,on='card_id',how='inner')
trans_new_df.shape

(179986, 230)

In [25]:
pickle.dump(trans_new_df,open('trans_new_df.pkl','wb'))

## 4.7 Create Date related boolean features

In [27]:
start_time = time.clock()
s_new_df['purchase_date'] = pd.to_datetime(s_new_df['purchase_date'])
s_new_df['day'] = s_new_df['purchase_date'].dt.day
s_new_df['wday'] = s_new_df['purchase_date'].dt.weekday
s_new_df['month'] = s_new_df['purchase_date'].dt.month
s_new_df['year'] = s_new_df['purchase_date'].dt.year
end_time = time.clock()
print("time taken in seconds : ",end_time-start_time)

time taken in seconds :  1.673487199999954


In [28]:
temp_df = eda_fe.createDateRelatedBoolFeat(s_new_df)

Creating is_purchase_month_end...
Creating is_purchase_month_start...
Creating is_purchase_quarter_start...
Creating is_purchase_quarter_end...
Creating is_purchase_year_end...
Creating is_purchase_year_start...
Creating is_christmas...
Creating is_mothers_day...
Creating is_childrens_day...
Creating is_valentines_day...
Creating is_fathers_day...
Time taken for completion :  500.2914159999998


In [33]:
bool_datetime_cols = [col for col in s_new_df.columns if col.startswith('is_')]
stat_func = ['mode','min','count','nunique']
for col in tqdm(bool_datetime_cols):
    for func in tqdm(stat_func): 
        trans_new_df = pd.merge(trans_new_df,eda_fe.computeCategoricalAggr(s_new_df,col,func),on='card_id',how='inner')

100%|███████████████████████████████████████████████████████████████████████████████| 11/11 [1:01:05<00:00, 333.22s/it]


In [35]:
pickle.dump(trans_new_df,open('trans_new_df.pkl','wb'))

## Summary of the notebook

### Strategy to follow
* Perform feature transformation on each of the dataframe(train_data,historical_transactions,new_merchant_transactions and merchants_data,
new_merchant_transactions is same as historical_transactions with card_ids from test_df) and merge them all in the end.
* Also selective sampling of the data from historical_transactions,new_merchant_transactions and merchants_df has to be done to form the train data.
* After the sampling,transformation and imputation of missing data is done, we merge the transformed data.

### Train_data
* Performed transformation of train_data,apart from whatever was done in EDA, seperate bins were formed for feature_1 and feature_2 based on the observations of box-plots in train_df.
* Imputations were done for category_2 and category_3 based on their modal/max occuring category.

### Historical_transactions
* Computing Statistical aggregation over categorical data.
* Created date-time related features.
* Created purchase amount features with aggregation over different month_lags.
* Created purchase amount ratios wrt above features.
#### ratios were calculated as follows:
	 * np_amount_min_0_2 = purch_lag_df['np_amount_min_0']/purch_lag_df['np_amount_min_2']
	 * where 0 represents no time lag and 2 represents a shift of -2
	 * ratios were calculated over a lag-shift of -1 and -2
* Created window averages wrt above features.(window_size=2,3)
* Created features related to installments.
* Created tfidf and word2vec features for merchant_category_id and subsector_id.
	* Applied svd on tfidf vectors and reduced the size of vectors to 40 and 10 respectively which explains about 75% of the variance in data.
	* The size of the word2vec vectors is 30 and 10 respectively.
* Create purchase_date related boolean features.

### New Merchants transactions data
* Applied same set of transformation as for historical transactions.

#### Merging of all the above generated dataframes and filling up of the missing values happens in the next notebook Feature Imputation.ipynb